<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AWS RDS - Configuration</title>
    <style>
        h1 {
            color: #00bfff; /* Light blue color */
        }
    </style>
</head>
<body>
    <h1>AWS RDS - Configuration</h1>
    <p>This notebook shows the configuration on an AWS RDS:</p>
    <ul>
        <li>RDS object configuration: CPU, memory, IO... The Notebooks needs AWS Credential to retrieve the information</li>
        <li>Postgres Settings - Max connections, logs, query cost settings... The Notebook needs a connection string to the DB to get the data from the table pg_catalog.pg_settings</li>
        <li>The configured extensions</li>
    </ul>
    <p>This early version of a notebook brings the data as is, without any analysis of the information.</p>
</body>
</html>


## Configuration
- Configure the AWS credentials: access_key_id, secret_access_key, region_name
- Configure the PG connections string

In [2]:
import configparser

# Read from the Config file
try: 
    config = configparser.ConfigParser() 
    config.read_file(open(r'../ipynb.cfg'))
except Exception as e:
    print(f"Error opening the configuration file: {e}")

try: 
    # con_str = config.get('con_str', 'PG_AIRBASES')
    access_key_id = config.get('credentials', 'ACCESS_KEY_ID') 
    secret_access_key = config.get('credentials', 'SECRET_ACCESS_KEY')   
    region_name =  'eu-central-1' 
except Exception as e:
    print(f"Error opening the configuration file: {e}")



## Get a list of RDS Instances
The AWS credentials might manage many RDS instances. 

In [3]:
import boto3

def create_aws_session(access_key_id, secret_access_key, region_name):
    try:
        # Create a session using AWS credentials
        session = boto3.Session(
            aws_access_key_id=access_key_id,
            aws_secret_access_key=secret_access_key,
            region_name=region_name
        )
        return session
    except Exception as e:
        print("An error occurred while creating the AWS session:", str(e))
        return None

def list_rds_instances(session):
    try:
        if session is None:
            print("AWS session is not valid.")
            return []

        # Create an RDS client using the session
        rds_client = session.client('rds')

        # Retrieve the RDS instances
        response = rds_client.describe_db_instances()

        # Get the RDS instance information
        rds_instances = response['DBInstances']
        return rds_instances

    except Exception as e:
        print("An error occurred while listing RDS instances:", str(e))
        return []

session = create_aws_session(access_key_id, secret_access_key, region_name)
rds_instances = list_rds_instances(session)

if len(rds_instances) > 0:
    print("RDS Instances:")
    for instance in rds_instances:
        instance_id = instance['DBInstanceIdentifier']
        instance_class = instance['DBInstanceClass']
        endpoint = instance['Endpoint']['Address']
        print("Instance ID:", instance_id)
        print("Instance Class:", instance_class)
        print("Endpoint:", endpoint)
        print("---")
else:
    print("No RDS instances found.")


RDS Instances:
Instance ID: database-2
Instance Class: db.m6g.large
Endpoint: database-2.cofhrj7zmyn4.eu-central-1.rds.amazonaws.com
---
Instance ID: db-4-v14
Instance Class: db.t4g.micro
Endpoint: db-4-v14.cofhrj7zmyn4.eu-central-1.rds.amazonaws.com
---
Instance ID: db-v-137
Instance Class: db.t4g.micro
Endpoint: db-v-137.cofhrj7zmyn4.eu-central-1.rds.amazonaws.com
---
Instance ID: metis-prod-v2
Instance Class: db.m6g.large
Endpoint: metis-prod-v2.cofhrj7zmyn4.eu-central-1.rds.amazonaws.com
---
Instance ID: mysql-for-test
Instance Class: db.t3.micro
Endpoint: mysql-for-test.cofhrj7zmyn4.eu-central-1.rds.amazonaws.com
---


## Show the Details of an RDS Instance
Using Boto3, to get the data from the RDS object. 

TODO: Calculate the actual CPU, memory and IO from the instance type (such as  db.m6g.large)

In [4]:
import boto3
import pandas as pd

def create_aws_session(access_key_id, secret_access_key, region_name):
    try:
        # Create a session using AWS credentials
        session = boto3.Session(
            aws_access_key_id=access_key_id,
            aws_secret_access_key=secret_access_key,
            region_name=region_name
        )
        return session
    except Exception as e:
        print("An error occurred while creating the AWS session:", str(e))
        return None

# Shows the details of an RDS instance 
def rds_details(rds_name, session):
    try:
        if session is None:
            print("AWS session is not valid.")
            return None

        # Create an RDS client using the session
        rds_client = session.client('rds')

        # Retrieve the RDS instance details
        response = rds_client.describe_db_instances(DBInstanceIdentifier=rds_name)

        if 'DBInstances' in response:
            rds_instance = response['DBInstances'][0]

            # Convert the instance details to a pandas DataFrame
            df = pd.DataFrame.from_dict(rds_instance, orient='index', columns=['Value'])
            df.index.name = 'Property'

            return df
        else:
            print("RDS instance not found:", rds_name)

    except Exception as e:
        print("An error occurred while retrieving RDS instance details:", str(e))

    return None



session = create_aws_session(access_key_id, secret_access_key, region_name)
#rds_instances = list_rds_instances(session)


rds_name = "database-2"
df = rds_details(rds_name, session)

if df is not None:
    print("RDS Instance Details:")
    print(df)

    # Save DataFrame to a file (e.g., CSV)
    # df.to_csv("rds_instance_details.csv", index=True)
    # print("--------------------------------------")
    # print("DataFrame saved to 'rds_instance_details.csv'.")
else:
    print("No RDS instance details available.")

RDS Instance Details:
                                                                                Value
Property                                                                             
DBInstanceIdentifier                                                       database-2
DBInstanceClass                                                          db.m6g.large
Engine                                                                       postgres
DBInstanceStatus                                                            available
MasterUsername                                                               postgres
Endpoint                            {'Address': 'database-2.cofhrj7zmyn4.eu-centra...
AllocatedStorage                                                                  239
InstanceCreateTime                                   2022-03-24 17:25:30.255000+00:00
PreferredBackupWindow                                                     21:43-22:13
BackupRetentionPeriod           